# Lesson 07 — A2A Client Fundamentals

Build a client that **discovers** and exercises **all A2A protocol capabilities**
of the enhanced InsuranceAgent server.

## What You'll Learn

- Resolve an Agent Card and inspect **multiple skills**
- Send **blocking** requests with `A2AClient.send_message()`
- Handle **multi-turn conversations** (`input_required` state)
- **Stream** responses with `A2AClient.send_message_streaming()`
- Parse **Artifacts** with `TextPart` and `DataPart` content
- **Cancel** tasks via `tasks/cancel`
- Exchange **structured data** (JSON artifacts)
- Handle **errors** gracefully (JSON-RPC error codes)

## A2A Protocol Features Exercised

| Feature         | Client Method                  | What to Observe                              |
| --------------- | ------------------------------ | -------------------------------------------- |
| Discovery       | `A2ACardResolver`              | 3 skills, streaming + stateTransitionHistory |
| Blocking Q&A    | `send_message()`               | Text response (Message)                      |
| Multi-Turn      | `send_message()` with `taskId` | `input_required` → follow-up → `completed`   |
| Streaming       | `send_message_streaming()`     | Working → output → completed events          |
| Artifacts       | Response parsing               | `DataPart` in claim receipts and summaries   |
| Task Cancel     | Raw JSON-RPC `tasks/cancel`    | `canceled` state                             |
| Structured Data | `DataPart` artifacts           | Machine-readable JSON                        |
| Error Handling  | Error response parsing         | JSON-RPC error codes                         |

## Prerequisites

- **Lesson 06 server must be running** on `http://localhost:10001`
- Open `../06-a2a-server/src/a2a_server.ipynb` and run all cells through Step 6

> **A2A SDK docs:** [pypi.org/project/a2a-sdk](https://pypi.org/project/a2a-sdk/)


## How the A2A Client Connects to the Server

```mermaid
graph LR
    subgraph Client["Lesson 07 — This Notebook"]
        Resolver["A2ACardResolver\n(discovery)"]
        A2AC["A2AClient\n(send_message / send_message_streaming)"]
        httpx["httpx.AsyncClient\n(underlying HTTP transport)"]
        A2AC --> httpx
    end

    subgraph Server["Lesson 06 — InsuranceAgent Server (port 10001)"]
        WellKnown[".well-known/agent.json\n(Agent Card)"]
        JSONRPC["JSON-RPC endpoint /\n(message/send, message/stream,\ntasks/get, tasks/cancel)"]
    end

    Resolver -->|GET /.well-known/agent.json| WellKnown
    httpx -->|POST /| JSONRPC

    style Client fill:#e9f7ef,stroke:#27ae60
    style Server fill:#fef9e7,stroke:#f39c12
```

**Two phases in every client interaction:**

1. **Discovery** — `A2ACardResolver` fetches the Agent Card once. You learn what skills exist, whether streaming is supported, and what content types the agent accepts.
2. **Execution** — `A2AClient` sends `message/send` (blocking) or `message/stream` (SSE) requests. Both use the same JSON-RPC wire format — only the method name differs.


## Step 1 — Install Dependencies


In [33]:
%pip install "a2a-sdk[http-server]" httpx python-dotenv --quiet
print("Dependencies ready.")

Note: you may need to restart the kernel to use updated packages.
Dependencies ready.



[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [34]:
import json
import warnings
from uuid import uuid4

import httpx
from dotenv import find_dotenv, load_dotenv

env_path = find_dotenv(raise_error_if_not_found=False)
if env_path:
    load_dotenv(env_path)
    print(f"Loaded .env from: {env_path}")

warnings.filterwarnings("ignore", category=DeprecationWarning)

BASE_URL = "http://localhost:10001"
print(f"Target server: {BASE_URL}")
print("NOTE: Lesson 06 server must be running on this address")

Loaded .env from: y:\.sources\localm-tuts\a2a\_examples\.env
Target server: http://localhost:10001
NOTE: Lesson 06 server must be running on this address


## Step 2 — Discover the Agent (Rich Agent Card)

Every A2A agent serves its Agent Card at `/.well-known/agent.json`.
Our enhanced server declares **3 skills** and **streaming + stateTransitionHistory**.


In [35]:
from a2a.client import A2ACardResolver

# timeout=60.0 — model inference can take 20-40s
async with httpx.AsyncClient(timeout=60.0) as temp_client:
    resolver = A2ACardResolver(
        httpx_client=temp_client,
        base_url=BASE_URL,
    )
    agent_card = await resolver.get_agent_card()

# ── Inspect the Agent Card ────────────────────────────────────────────────
print(f"Agent Name:        {agent_card.name}")
print(f"Description:       {agent_card.description}")
print(f"Version:           {agent_card.version}")
print(f"URL:               {agent_card.url}")
print()

# Capabilities — Pydantic fields use snake_case for Python attribute access
caps = agent_card.capabilities
print("Capabilities:")
print(f"  Streaming:                {caps.streaming}")
print(f"  State Transition History: {caps.state_transition_history}")
print(f"  Push Notifications:       {caps.push_notifications}")
print()

# Output modes
print(f"Input Modes:  {agent_card.default_input_modes}")
print(f"Output Modes: {agent_card.default_output_modes}")
print()

# Skills
print(f"Skills ({len(agent_card.skills)}):")
for skill in agent_card.skills:
    print(f"  [{skill.id}] {skill.name}")
    print(f"    Description: {skill.description}")
    print(f"    Tags: {skill.tags}")
    print(f"    Examples: {skill.examples}")
    print()

Agent Name:        InsuranceAgent
Description:       Multi-skill insurance agent using Phi-4 via github. Handles policy Q&A, claims filing (multi-turn), and structured policy summaries.
Version:           2.0.0
URL:               http://localhost:10001/

Capabilities:
  Streaming:                True
  State Transition History: True
  Push Notifications:       None

Input Modes:  ['text']
Output Modes: ['text', 'data']

Skills (3):
  [policy-qa] Policy Question Answering
    Description: Answer questions about insurance policy coverage, deductibles, premiums, and exclusions
    Tags: ['qa', 'insurance', 'policy', 'coverage']
    Examples: ['What is the deductible for the Standard plan?', 'Are cosmetic procedures covered?', 'What medications are excluded?']

  [claims-filing] Claims Filing
    Description: File insurance claims through a multi-turn conversation. Collects claim type, date, amount, and description. Returns a structured claim receipt as a DataPart artifact.
    Tags: ['cla

## Step 3 — Create A2A Client + Helpers

Set up the A2A client and helper functions for building requests
and parsing responses.


In [36]:
from a2a.client import A2AClient
from a2a.types import MessageSendParams, SendMessageRequest, SendStreamingMessageRequest

# Persistent client for all requests
httpx_client = httpx.AsyncClient(timeout=120.0)

client = A2AClient(
    httpx_client=httpx_client,
    agent_card=agent_card,
)

print(f"A2AClient ready — targeting {agent_card.url}")

A2AClient ready — targeting http://localhost:10001/


In [37]:
def build_request(
    text: str,
    task_id: str | None = None,
    context_id: str | None = None,
) -> SendMessageRequest:
    """Build a blocking SendMessageRequest.

    Args:
        text: User message text.
        task_id: Set for follow-up messages to an existing task.
        context_id: Set to group related tasks in a context.
    """
    payload = {
        "message": {
            "role": "user",
            "parts": [{"kind": "text", "text": text}],
            "messageId": uuid4().hex,
        }
    }
    if task_id:
        payload["taskId"] = task_id
    if context_id:
        payload["contextId"] = context_id

    return SendMessageRequest(
        id=str(uuid4()),
        params=MessageSendParams(**payload),
    )


def build_streaming_request(
    text: str,
    task_id: str | None = None,
) -> SendStreamingMessageRequest:
    """Build a streaming SendStreamingMessageRequest."""
    payload = {
        "message": {
            "role": "user",
            "parts": [{"kind": "text", "text": text}],
            "messageId": uuid4().hex,
        }
    }
    if task_id:
        payload["taskId"] = task_id

    return SendStreamingMessageRequest(
        id=str(uuid4()),
        params=MessageSendParams(**payload),
    )


print("Request builders defined.")

Request builders defined.


In [38]:
def extract_text(response) -> str:
    """Extract text from a Message or Task response."""
    try:
        result = response.root.result
    except AttributeError:
        return f"Error: unexpected response — {type(response).__name__}"

    texts = []

    # Case 1: Direct Message (v0.3.x message/send)
    if hasattr(result, "parts") and result.parts:
        for part in result.parts:
            pr = getattr(part, "root", part)
            if getattr(pr, "kind", None) == "text":
                texts.append(pr.text)

    # Case 2: Task with status message
    if hasattr(result, "status") and result.status:
        if result.status.message and result.status.message.parts:
            for part in result.status.message.parts:
                pr = getattr(part, "root", part)
                if getattr(pr, "kind", None) == "text":
                    texts.append(pr.text)

    return "\n".join(texts) if texts else "(no text in response)"


def get_task_info(response) -> dict:
    """Extract task state, ID, and artifacts from a response."""
    try:
        result = response.root.result
    except AttributeError:
        return {"error": f"Unexpected response: {type(response).__name__}"}

    info = {"type": type(result).__name__}

    # Task ID
    if hasattr(result, "id"):
        info["task_id"] = result.id
    if hasattr(result, "taskId"):
        info["task_id"] = result.taskId

    # Context ID
    if hasattr(result, "contextId"):
        info["context_id"] = result.contextId

    # Task state
    if hasattr(result, "status") and result.status:
        info["state"] = (
            result.status.state.value
            if hasattr(result.status.state, "value")
            else str(result.status.state)
        )

    # Artifacts
    if hasattr(result, "artifacts") and result.artifacts:
        artifacts = []
        for artifact in result.artifacts:
            a_info = {
                "id": getattr(artifact, "artifactId", "?"),
                "name": getattr(artifact, "name", "?"),
            }
            # Extract parts
            if hasattr(artifact, "parts") and artifact.parts:
                for part in artifact.parts:
                    pr = getattr(part, "root", part)
                    if getattr(pr, "kind", None) == "data":
                        a_info["data"] = pr.data
                    elif getattr(pr, "kind", None) == "text":
                        a_info["text"] = pr.text
            artifacts.append(a_info)
        info["artifacts"] = artifacts

    # Message text
    if hasattr(result, "parts") and result.parts:
        for part in result.parts:
            pr = getattr(part, "root", part)
            if getattr(pr, "kind", None) == "text":
                info["text"] = pr.text
                break

    return info


print("Response parsers defined (extract_text, get_task_info).")

Response parsers defined (extract_text, get_task_info).


---

## Step 4 — Blocking Call: Policy Q&A

The simplest A2A interaction: send a question, get back a text answer.
This exercises the `policy-qa` skill.


In [39]:
request = build_request("What is the annual deductible?")
response = await client.send_message(request)

info = get_task_info(response)
print(f"Response type: {info.get('type')}")
print(f"Task state:    {info.get('state', 'n/a (Message)')}")
print(f"Task ID:       {info.get('task_id', 'n/a')}")
print()
print(f"Answer: {extract_text(response)}")

Response type: Task
Task state:    completed
Task ID:       7886ea4c-28cc-4b70-a0fd-6895a4a28059

Answer: The policy document does not specify a single "annual deductible." Instead, it outlines different deductibles for various services:

- Standard Plan Deductible: $500 per incident
- Emergency Room Deductible: $250 per visit
- Prescription Drug Deductible: $100 per year

Each of these deductibles applies to different types of covered services. For example, the Prescription Drug Deductible is the closest to an "annual deductible" for prescription drugs, as it is $100 per year. 

For more detailed information, you may refer to the "DEDUCTIBLES" section of the policy document.


In [40]:
# Test multiple Q&A questions
questions = [
    "What is the monthly premium?",
    "Are cosmetic procedures covered?",
    "What is the meaning of life?",  # out of scope
]

for q in questions:
    req = build_request(q)
    resp = await client.send_message(req)
    answer = extract_text(resp)
    print(f"Q: {q}")
    print(f"A: {answer[:200]}{'...' if len(answer) > 200 else ''}")
    print("-" * 60)

Q: What is the monthly premium?
A: The monthly premium for the Standard Plan is $150. This information is found in the "COVERAGE SUMMARY" section of the policy document.
------------------------------------------------------------
Q: Are cosmetic procedures covered?
A: Cosmetic procedures are not covered under the ACME Insurance Standard Policy, except when they are medically necessary. This is specified in the "EXCLUSIONS" section of the policy document. 

Relevant...
------------------------------------------------------------
Q: What is the meaning of life?
A: The question of the meaning of life is a philosophical and existential inquiry that has been explored by thinkers, philosophers, and theologians throughout history. It is not something that can be def...
------------------------------------------------------------


---

## Step 5 — Multi-Turn Conversation: Claims Filing

This is the most important A2A protocol feature to exercise.
The `claims-filing` skill uses **multi-turn** conversations:

1. **Turn 1**: Client sends partial claim info
2. **Server**: Returns task with `state: input_required` + message asking for missing fields
3. **Turn 2**: Client sends follow-up with `taskId` parameter to continue the same task
4. **Server**: Returns completed task with a `DataPart` artifact (claim receipt)

The key protocol mechanism:

- `TaskState.input_required` tells the client the agent needs more info
- The `taskId` parameter in the follow-up links it to the existing task
- The server maintains session state via `InMemoryTaskStore`


### Multi-Turn Protocol Flow

```mermaid
sequenceDiagram
    participant Client as A2AClient (you)
    participant Handler as DefaultRequestHandler
    participant Store as InMemoryTaskStore
    participant Executor as InsuranceAgentExecutor

    Note over Client,Executor: Turn 1 — no taskId (new task)

    Client->>Handler: message/send {message: "file a dental claim"}
    Handler->>Store: create Task(id=T1, state=working)
    Handler->>Executor: execute(context, queue)
    Executor->>Executor: detect_skill → claims-filing
    Executor->>Executor: ClaimsAgent.process() → input_required
    Executor-->>Handler: enqueue TaskStatusUpdateEvent(input_required)
    Handler->>Store: update Task(id=T1, state=input_required)
    Handler-->>Client: Task {id:"T1", state:"input_required", message:"Please provide: date, amount, description"}

    Note over Client: Client saves task_id="T1"

    Note over Client,Executor: Turn 2 — taskId="T1" (continue existing task)

    Client->>Handler: message/send {taskId:"T1", message:"root canal $450 2025-01-15"}
    Handler->>Store: load Task(id=T1) → input_required
    Handler->>Executor: execute(context, queue)  [context.task_id = "T1"]
    Executor->>Executor: ClaimsAgent.process(session_id="T1") → completed
    Executor-->>Handler: enqueue TaskArtifactUpdateEvent(DataPart receipt)
    Executor-->>Handler: enqueue TaskStatusUpdateEvent(completed, final=True)
    Handler->>Store: update Task(id=T1, state=completed, artifacts=[receipt])
    Handler-->>Client: Task {id:"T1", state:"completed", artifacts:[{name:"Claim Receipt", data:{...}}]}
```

**What to observe in the code below:**

- Turn 1: `task_id = turn1_info["task_id"]` — you must save this
- Turn 2: `build_request("...", task_id=task_id)` — you must pass it back
- Final response: `artifacts[0]["data"]` contains the claim receipt JSON


In [41]:
# ── Turn 1: Partial claim info ────────────────────────────────────────────
print("═" * 60)
print("MULTI-TURN CLAIMS FILING")
print("═" * 60)
print()

turn1_request = build_request("I need to file a dental claim")
turn1_response = await client.send_message(turn1_request)

turn1_info = get_task_info(turn1_response)
print("── Turn 1 ──")
print(f"Type:    {turn1_info.get('type')}")
print(f"State:   {turn1_info.get('state', 'n/a')}")
print(f"Task ID: {turn1_info.get('task_id', 'n/a')}")
print(f"Text:    {extract_text(turn1_response)}")
print()

# Save the task ID for the follow-up
claim_task_id = turn1_info.get("task_id")
print(f"→ Saved task_id for follow-up: {claim_task_id}")

════════════════════════════════════════════════════════════
MULTI-TURN CLAIMS FILING
════════════════════════════════════════════════════════════

── Turn 1 ──
Type:    Task
State:   input-required
Task ID: a0baaf70-0178-4312-8dfa-0879358a5966
Text:    Please provide: date_of_service, amount, description

→ Saved task_id for follow-up: a0baaf70-0178-4312-8dfa-0879358a5966


In [42]:
# ── Turn 2: Provide missing details (with taskId) ─────────────────────────
if claim_task_id:
    turn2_request = build_request(
        "Root canal on 2025-01-15 for $450",
        task_id=claim_task_id,  # ← links to existing task
    )
    turn2_response = await client.send_message(turn2_request)

    turn2_info = get_task_info(turn2_response)
    print("── Turn 2 ──")
    print(f"Type:    {turn2_info.get('type')}")
    print(f"State:   {turn2_info.get('state', 'n/a')}")
    print(f"Text:    {extract_text(turn2_response)}")
    print()

    # Check for artifacts (claim receipt)
    if "artifacts" in turn2_info:
        print("── Artifacts ──")
        for artifact in turn2_info["artifacts"]:
            print(f"  Name: {artifact.get('name', '?')}")
            if "data" in artifact:
                print(f"  Data (DataPart):")
                print(f"  {json.dumps(artifact['data'], indent=4)}")
            if "text" in artifact:
                print(f"  Text: {artifact['text']}")
    else:
        print("  (no artifacts in response)")
else:
    print("ERROR: No task_id from Turn 1 — cannot send follow-up")

── Turn 2 ──
Type:    Task
State:   completed
Text:    For a root canal procedure on January 15, 2025, costing $450, here's how the coverage would work under the ACME Insurance Standard Policy:

1. **Deductible**: Since a root canal is a dental procedure, it would typically fall under specialist visits. The Standard Plan Deductible is $500 per incident. Therefore, you would need to pay the full $450 out-of-pocket because it does not exceed the deductible.

2. **Co-insurance**: After the deductible is met, specialist visits are covered at 70/30 co-insurance. However, since the cost of the root canal ($450) does not exceed the deductible, co-insurance does not apply in this case.

3. **Coverage Limits**: The procedure cost is well within the per incident maximum of $50,000 and the annual maximum of $200,000.

In summary, you would be responsible for the full $450 cost of the root canal procedure because it does not exceed the $500 deductible for the incident. 

Relevant sections:
- Deduc

### Multi-Turn Flow Summary

```
Client                          Server
  │                               │
  │─── message/send (no taskId) ──→│  Turn 1: "file a dental claim"
  │                               │  → detect skill: claims-filing
  │                               │  → extract: claim_type=dental
  │                               │  → missing: date, amount, description
  │←── Task(input_required) ──────│  → emit: input_required + message
  │                               │
  │─── message/send (taskId) ────→│  Turn 2: "root canal 2025-01-15 $450"
  │                               │  → extract remaining fields
  │                               │  → all fields present → process
  │←── Task(completed) + Artifact─│  → emit: artifact + completed
  │                               │
```


---

## Step 6 — Structured Data: Policy Summary

The `policy-summary` skill returns a **DataPart artifact** — machine-readable
JSON that can be consumed by other agents or UIs.

In the A2A protocol:

- `TextPart` → human-readable text
- `DataPart` → structured JSON (the `data` field contains arbitrary JSON)
- `FilePart` → binary file references


In [43]:
summary_request = build_request("Give me a summary of my policy")
summary_response = await client.send_message(summary_request)

summary_info = get_task_info(summary_response)
print(f"Type:  {summary_info.get('type')}")
print(f"State: {summary_info.get('state', 'n/a')}")
print(f"Text:  {extract_text(summary_response)}")
print()

# Extract the structured data artifact
if "artifacts" in summary_info:
    for artifact in summary_info["artifacts"]:
        print(f"Artifact: {artifact.get('name', '?')}")
        if "data" in artifact:
            print("DataPart (structured JSON):")
            print(json.dumps(artifact["data"], indent=2))
        else:
            print(f"  Text: {artifact.get('text', '?')}")
else:
    print("(no artifacts — the summary may be in the message text)")

Type:  Task
State: completed
Text:  Policy summary generated.

Artifact: Policy Summary
DataPart (structured JSON):
{
  "policy_number": "ACME-STD-2025-001",
  "plan_name": "Standard",
  "monthly_premium": "$150",
  "annual_premium": "$1,800",
  "deductible": {
    "standard_plan": "$500 per incident",
    "emergency_room": "$250 per visit",
    "prescription_drug": "$100 per year"
  },
  "annual_maximum": "$200,000",
  "covered_services": [
    "Primary Care Visits: Covered after deductible, 80/20 co-insurance",
    "Specialist Visits: Covered after deductible, 70/30 co-insurance",
    "Emergency Room: Covered after ER deductible, 90/10 co-insurance",
    "Prescription Drugs: Generic: $10 co-pay, Brand Name: $30 co-pay, Specialty: 20% co-insurance after drug deductible",
    "Preventive Care: Covered at 100%, no deductible",
    "Mental Health: Covered after deductible, 80/20 co-insurance",
    "Physical Therapy: Up to 30 visits per year, $25 co-pay per visit"
  ],
  "exclusions": [
 

---

## Step 7 — Streaming Responses

Streaming returns partial results as Server-Sent Events (SSE).
With the enhanced server, you'll see **multiple event types**:

1. `TaskStatusUpdateEvent` with `state: working`
2. `Message` with the agent's answer OR `TaskArtifactUpdateEvent` with data
3. `TaskStatusUpdateEvent` with `state: completed`

This is critical for real-time UIs — show a spinner when `working`,
render results as they arrive, and update status on `completed`.


### Streaming Event Flow (Server-Sent Events)

```mermaid
sequenceDiagram
    participant Client as A2AClient (async for)
    participant Server as A2A Server (SSE stream)

    Client->>Server: POST / {method: "message/stream", ...}
    Note over Server: Opens SSE connection, does NOT wait for completion

    Server-->>Client: data: TaskStatusUpdateEvent(state=working)\n
    Note over Client: event #1 — show spinner

    Server-->>Client: data: Message(TextPart("answer text"))\n   (Q&A)
    Note over Client: event #2 — render text as it arrives

    Server-->>Client: data: TaskStatusUpdateEvent(state=completed, final=true)\n
    Note over Client: event #3 — hide spinner, finalize

    Note over Server,Client: OR for structured data (policy-summary):

    Server-->>Client: data: TaskStatusUpdateEvent(state=working)\n
    Server-->>Client: data: TaskArtifactUpdateEvent(DataPart{...})\n
    Note over Client: event — extract artifact["data"], render JSON
    Server-->>Client: data: TaskStatusUpdateEvent(state=completed, final=true)\n
```

**Key difference from blocking calls:**
With `send_message()` you wait for the complete response. With `send_message_streaming()`
you get each event as it is emitted by the server — you can progressively render the UI.
The `async for event in client.send_message_streaming(req):` pattern iterates over this SSE stream.


In [44]:
# ── Stream a Q&A request ──────────────────────────────────────────────────
streaming_request = build_streaming_request("Explain the claims process step by step.")

print("Streaming Q&A response:")
print("═" * 60)

event_count = 0
async for event in client.send_message_streaming(streaming_request):
    event_count += 1
    event_type = type(event).__name__
    print(f"\n  Event #{event_count}: {event_type}")

    root = getattr(event, "root", event)
    result = getattr(root, "result", None)
    if result is None:
        continue

    # Check for task status updates
    if hasattr(result, "status") and result.status:
        state = result.status.state
        state_val = state.value if hasattr(state, "value") else str(state)
        print(f"  State: {state_val}")
        if result.status.message:
            for part in result.status.message.parts:
                pr = getattr(part, "root", part)
                if getattr(pr, "kind", None) == "text":
                    preview = pr.text[:150].replace("\n", " ")
                    print(f"  Message: {preview}{'...' if len(pr.text) > 150 else ''}")

    # Check for artifacts
    if hasattr(result, "artifacts") and result.artifacts:
        for artifact in result.artifacts:
            print(f"  Artifact: {getattr(artifact, 'name', '?')}")

    # Check for direct message (parts on result)
    if hasattr(result, "parts") and result.parts:
        for part in result.parts:
            pr = getattr(part, "root", part)
            if getattr(pr, "kind", None) == "text":
                preview = pr.text[:150].replace("\n", " ")
                print(f"  Text: {preview}{'...' if len(pr.text) > 150 else ''}")

print("\n" + "═" * 60)
print(f"Streaming complete ({event_count} events).")

Streaming Q&A response:
════════════════════════════════════════════════════════════

  Event #1: SendStreamingMessageResponse
  State: working
  Message: Processing with policy-qa skill...

  Event #2: SendStreamingMessageResponse
  State: completed
  Message: Certainly! Here is the step-by-step claims process as outlined in the policy document:  1. **Submit Claims Within 90 Days of Service**:     - You must...

════════════════════════════════════════════════════════════
Streaming complete (2 events).


In [45]:
# ── Stream a policy summary (structured artifact) ─────────────────────────
stream_summary_req = build_streaming_request("Give me a policy summary")

print("Streaming structured data response:")
print("═" * 60)

event_count = 0
async for event in client.send_message_streaming(stream_summary_req):
    event_count += 1
    event_type = type(event).__name__
    print(f"\n  Event #{event_count}: {event_type}")

    root = getattr(event, "root", event)
    result = getattr(root, "result", None)
    if result is None:
        continue

    if hasattr(result, "status") and result.status:
        state_val = getattr(result.status.state, "value", str(result.status.state))
        print(f"  State: {state_val}")

    if hasattr(result, "artifacts") and result.artifacts:
        for artifact in result.artifacts:
            name = getattr(artifact, "name", "?")
            print(f"  Artifact: {name}")
            if hasattr(artifact, "parts") and artifact.parts:
                for part in artifact.parts:
                    pr = getattr(part, "root", part)
                    if getattr(pr, "kind", None) == "data":
                        print(f"  DataPart: {json.dumps(pr.data, indent=2)[:300]}...")

print("\n" + "═" * 60)
print(f"Streaming complete ({event_count} events).")

Streaming structured data response:
════════════════════════════════════════════════════════════

  Event #1: SendStreamingMessageResponse
  State: working

  Event #2: SendStreamingMessageResponse

  Event #3: SendStreamingMessageResponse
  State: completed

════════════════════════════════════════════════════════════
Streaming complete (3 events).


---

## Step 8 — Task Cancellation

The A2A protocol supports `tasks/cancel` for stopping in-progress tasks.
This is particularly useful for multi-turn conversations where the user
changes their mind.

**Flow:**

1. Start a claims filing (gets `input_required` state)
2. Instead of following up, cancel the task
3. Server cleans up the claims session

The SDK's `A2AClient` may not have a built-in `cancel_task()` method,
so we use raw `httpx` with the JSON-RPC payload. This is educational —
it shows the wire protocol directly.


In [46]:
# ── Start a claim (to get a task to cancel) ───────────────────────────────
cancel_req = build_request("I want to file a medical claim")
cancel_resp = await client.send_message(cancel_req)

cancel_info = get_task_info(cancel_resp)
cancel_task_id = cancel_info.get("task_id")

print(f"Started claim task: {cancel_task_id}")
print(f"State: {cancel_info.get('state', 'n/a')}")
print(f"Message: {extract_text(cancel_resp)}")
print()

if cancel_task_id:
    # ── Cancel the task via raw JSON-RPC ──────────────────────────────────
    cancel_payload = {
        "jsonrpc": "2.0",
        "id": str(uuid4()),
        "method": "tasks/cancel",
        "params": {
            "id": cancel_task_id,
        },
    }

    cancel_http_resp = await httpx_client.post(
        BASE_URL,
        json=cancel_payload,
        headers={"Content-Type": "application/json"},
    )

    print("Cancel response:")
    try:
        cancel_result = cancel_http_resp.json()
        print(json.dumps(cancel_result, indent=2))
    except Exception as e:
        print(f"Status: {cancel_http_resp.status_code}")
        print(f"Body: {cancel_http_resp.text[:500]}")
else:
    print("No task ID available — cannot cancel")

Started claim task: 30bf8f87-590f-40ae-9ee1-c019eda54b1a
State: input-required
Message: Please provide: claim_type, date_of_service, amount, description

Cancel response:
{
  "id": "48c1ca6d-7b20-4d13-b365-2902a7cf3bf1",
  "jsonrpc": "2.0",
  "result": {
    "contextId": "525f9a31-1d4a-436e-8503-ab8069d290d3",
    "history": [
      {
        "contextId": "525f9a31-1d4a-436e-8503-ab8069d290d3",
        "kind": "message",
        "messageId": "9c62480c4a474903bd6a64029067b24f",
        "parts": [
          {
            "kind": "text",
            "text": "I want to file a medical claim"
          }
        ],
        "role": "user",
        "taskId": "30bf8f87-590f-40ae-9ee1-c019eda54b1a"
      },
      {
        "kind": "message",
        "messageId": "eb59d4925add403c844b523b5f19b449",
        "parts": [
          {
            "kind": "text",
            "text": "Processing with claims-filing skill..."
          }
        ],
        "role": "agent"
      },
      {
        "kind": "

---

## Step 9 — Task Polling (tasks/get)

The A2A protocol supports `tasks/get` to retrieve the current state
of a task by its ID. This is useful for:

- Polling for completion (alternative to streaming)
- Retrieving artifacts after the fact
- Inspecting state transition history


In [47]:
# Use a completed task ID from the claims filing above
# If claims filing completed, use claim_task_id; otherwise create a new task
poll_task_id = claim_task_id  # from Step 5

if poll_task_id:
    get_task_payload = {
        "jsonrpc": "2.0",
        "id": str(uuid4()),
        "method": "tasks/get",
        "params": {
            "id": poll_task_id,
        },
    }

    poll_resp = await httpx_client.post(
        BASE_URL,
        json=get_task_payload,
        headers={"Content-Type": "application/json"},
    )

    print(f"tasks/get for task {poll_task_id}:")
    try:
        poll_result = poll_resp.json()
        # Pretty print with truncation for readability
        formatted = json.dumps(poll_result, indent=2)
        if len(formatted) > 2000:
            print(formatted[:2000] + "\n... (truncated)")
        else:
            print(formatted)
    except Exception as e:
        print(f"Status: {poll_resp.status_code}")
        print(f"Body: {poll_resp.text[:500]}")
else:
    print("No task_id available for polling")

tasks/get for task a0baaf70-0178-4312-8dfa-0879358a5966:
{
  "id": "a18afccb-a33d-4805-9749-abf2fa78d839",
  "jsonrpc": "2.0",
  "result": {
    "contextId": "91378c24-7961-409f-a150-6b6f115df85d",
    "history": [
      {
        "contextId": "91378c24-7961-409f-a150-6b6f115df85d",
        "kind": "message",
        "messageId": "980fc1eb08644823b2918c68494f5fa4",
        "parts": [
          {
            "kind": "text",
            "text": "I need to file a dental claim"
          }
        ],
        "role": "user",
        "taskId": "a0baaf70-0178-4312-8dfa-0879358a5966"
      },
      {
        "kind": "message",
        "messageId": "c6defede18be47bf83381c09b1c21567",
        "parts": [
          {
            "kind": "text",
            "text": "Processing with claims-filing skill..."
          }
        ],
        "role": "agent"
      }
    ],
    "id": "a0baaf70-0178-4312-8dfa-0879358a5966",
    "kind": "task",
    "status": {
      "message": {
        "kind": "message",
  

---

## Step 10 — Error Handling

A2A uses JSON-RPC error codes. Common errors:

| Code   | Meaning             |
| ------ | ------------------- |
| -32700 | Parse error         |
| -32600 | Invalid request     |
| -32601 | Method not found    |
| -32001 | Task not found      |
| -32002 | Task not cancelable |


In [48]:
async def safe_query(client: A2AClient, text: str) -> str:
    """Send a message and handle errors gracefully."""
    try:
        request = build_request(text)
        response = await client.send_message(request)

        # Check for JSON-RPC error
        root = getattr(response, "root", None)
        if root and hasattr(root, "error") and root.error:
            code = getattr(root.error, "code", "?")
            msg = getattr(root.error, "message", str(root.error))
            return f"[Error {code}] {msg}"

        return extract_text(response)

    except httpx.ConnectError:
        return "[Connection Error] Is the server running on localhost:10001?"
    except httpx.ReadTimeout:
        return "[Timeout] Server took too long to respond"
    except Exception as e:
        return f"[{type(e).__name__}] {e}"


# Test with a normal question
answer = await safe_query(client, "What medications are excluded?")
print(f"Normal: {answer[:200]}")
print()

Normal: The policy document does not explicitly list specific medications that are excluded. However, it does mention that "Experimental treatments not FDA-approved" are excluded. This implies that medication



In [49]:
# ── Test: Get non-existent task ───────────────────────────────────────────
bad_task_payload = {
    "jsonrpc": "2.0",
    "id": str(uuid4()),
    "method": "tasks/get",
    "params": {
        "id": "non-existent-task-id-12345",
    },
}

error_resp = await httpx_client.post(
    BASE_URL,
    json=bad_task_payload,
    headers={"Content-Type": "application/json"},
)

print("Error response for non-existent task:")
print(json.dumps(error_resp.json(), indent=2))

Error response for non-existent task:
{
  "error": {
    "code": -32001,
    "message": "Task not found"
  },
  "id": "c217f418-bda4-4ff7-8963-56aff8483839",
  "jsonrpc": "2.0"
}


In [50]:
# ── Test: Invalid method ──────────────────────────────────────────────────
invalid_method_payload = {
    "jsonrpc": "2.0",
    "id": str(uuid4()),
    "method": "tasks/nonexistent",
    "params": {},
}

error_resp2 = await httpx_client.post(
    BASE_URL,
    json=invalid_method_payload,
    headers={"Content-Type": "application/json"},
)

print("Error response for invalid method:")
print(json.dumps(error_resp2.json(), indent=2))

Error response for invalid method:
{
  "error": {
    "code": -32601,
    "message": "Method not found"
  },
  "id": "fd6ed778-265a-4ebe-b8fa-2982764d68ab",
  "jsonrpc": "2.0"
}


---

## Step 11 — Experiment!

Try your own interactions. Some ideas:

- File a claim with ALL info in one message (single-turn)
- File a claim with info spread across 3 turns
- Ask Q&A and summary in the same session
- Stream a claims filing


In [51]:
# Try your own!
my_text = (
    "File a claim for a medical emergency on 2025-03-01 for $1200 — ambulance transport"
)

my_request = build_request(my_text)
my_response = await client.send_message(my_request)

my_info = get_task_info(my_response)
print(f"State: {my_info.get('state', 'n/a')}")
print(f"Text:  {extract_text(my_response)}")

if "artifacts" in my_info:
    for a in my_info["artifacts"]:
        print(f"\nArtifact: {a.get('name', '?')}")
        if "data" in a:
            print(json.dumps(a["data"], indent=2))

State: completed
Text:  Claim CLM-43D45C99 filed successfully.

Artifact: Claim Receipt — CLM-43D45C99
{
  "claim_id": "CLM-43D45C99",
  "policy_number": "ACME-STD-2025-001",
  "claim_type": "emergency",
  "date_of_service": "2025-03-01",
  "amount": "1200",
  "description": "ambulance transport",
  "status": "submitted",
  "filed_at": "2026-02-28T23:06:13.551552+00:00",
  "estimated_processing_days": 30
}


## Cleanup


In [ ]:
await httpx_client.aclose()
print("httpx client closed.")

httpx client closed.


: 

## Summary

| Concept             | What You Learned                                    |
| ------------------- | --------------------------------------------------- |
| **Discovery**       | `A2ACardResolver` fetches Agent Card with 3 skills  |
| **Blocking calls**  | `send_message()` for simple Q&A                     |
| **Multi-turn**      | `input_required` state + `taskId` follow-ups        |
| **Streaming**       | SSE events: working → output → completed            |
| **Artifacts**       | `DataPart` for structured JSON (receipts/summaries) |
| **Task management** | `tasks/get` for polling, `tasks/cancel` for cancel  |
| **Error handling**  | JSON-RPC error codes + connection error handling    |

## A2A Protocol Coverage

| Protocol Feature            | Demonstrated?                                    |
| --------------------------- | ------------------------------------------------ |
| Agent Card Discovery        | ✅ 3 skills, capabilities inspection             |
| message/send (blocking)     | ✅ Q&A, claims, summary                          |
| message/stream (streaming)  | ✅ Event-by-event parsing                        |
| Multi-Turn (input_required) | ✅ Claims filing with follow-up                  |
| Artifacts (DataPart)        | ✅ Claim receipts, policy summaries              |
| Artifacts (TextPart)        | ✅ Q&A answers                                   |
| tasks/get (polling)         | ✅ Retrieve task by ID                           |
| tasks/cancel                | ✅ Cancel in-progress claims                     |
| Task State Machine          | ✅ working → input_required → completed/canceled |
| Error Handling              | ✅ Invalid method, non-existent task             |

## Next Steps

You've completed the full A2A loop: **Agent → Server → Client**
with comprehensive protocol coverage.

In Lesson 08, you'll build a more complex agent using the
**Microsoft AutoGen / Agent Framework** — and expose it over A2A.

→ Continue to [Lesson 08 — Microsoft Agent Framework](../08-microsoft-agent-framework/)
